A script that retrieves the crawl top 1000 highest scores once per day, then produces some stats from the results.

Notes

- script is triggered by a daily cron job
- data is stored in sqlite3
- running once per day runs a small risk of missing a score that appeared briefly before being knocked off that day.

In [113]:
import pandas as pd
import time
import sqlite3

In [114]:
# get the date

print("Last updated " + time.strftime("%d/%m/%Y at %H:%M"))

Last updated 04/06/2021 at 13:57


In [115]:
# connect to the db

con = sqlite3.connect('crawl_scores.db')
cur = con.cursor()

In [116]:
# read the previous update into a dataframe

old_dcss_data = pd.read_sql('SELECT * FROM scores', con)

In [117]:
# store the last update into the old_scores table for comparison

old_dcss_data.to_sql("old_scores", con, if_exists="replace", index=False)

In [118]:
# get the hi score table from the crawl website into a dataframe

url = 'http://crawl.akrasiac.org/scoring/top-N.html'
tables = pd.read_html(url)
new_dcss_data = tables[0]

In [119]:
# clean up table and add unique ID from score and date

new_dcss_data = new_dcss_data.rename(columns = {'Unnamed: 0' : "Rank"})

In [120]:
# put the new dataframe into the sqlite3 db

# write the dataframe to the db
new_dcss_data.to_sql("scores", con, if_exists="replace", index=False)

In [121]:
# get any new entries

new_entries = pd.read_sql_query("SELECT * FROM scores EXCEPT SELECT * FROM old_scores", con)

In [122]:
# get knocked out entries

knocked_out = pd.read_sql_query("SELECT * FROM old_scores EXCEPT SELECT * FROM scores", con)

In [123]:
# save knocked out entries

knocked_out.to_sql("knocked", con, if_exists="append", index=False)

In [ ]:
# show last time scores changed
last_change = new_dcss_data["Date"].sort_values(ascending=False)
last_change = str(last_change.iloc[0])

print("Top 1000 last changed " + last_change)

In [153]:
# show total wins in prod

prod_scores = new_dcss_data[~new_dcss_data["Version"].str.contains("-")]
non_prod_scores = new_dcss_data[new_dcss_data["Version"].str.contains("-")]

print("Total wins from complete versions: " + str(len(prod_scores.index)))
print("Total wins from git versions: " + str(len(non_prod_scores.index)))

Total wins from complete versions: 323
Total wins from git versions: 677


In [ ]:
most_recent["Date"]

In [19]:
my_first_df = pd.read_pickle("20210529-151629")

In [20]:
my_second_df = pd.read_pickle("20210603-160607")

In [21]:
print(my_first_df)
print(my_second_df)

     Unnamed: 0     Score        Player Character              God  \
103         104  54029826          Wong      DEFE          Vehumet   
873         874  32473507     p0werm0de      GrFi  the Shining One   
11           12  82486239        Sergey      DEVM         Sif Muna   
840         841  32967403       Catclaw      FoDe           Lugonu   
634         635  35981393  Nebukadnezar      BaVM  the Shining One   

                   Title  Place                                 End  XL  \
103             Archmage    NaN  escaped with the Orb and 15 runes!  27   
873               Slayer    NaN  escaped with the Orb and 15 runes!  27   
11              Archmage    NaN  escaped with the Orb and 15 runes!  27   
840  Corrupter of Planes    NaN  escaped with the Orb and 15 runes!  27   
634       Petrodigitator    NaN  escaped with the Orb and 15 runes!  27   

     Turns  Duration  Runes                 Date  Version Server  
103  26817  03:07:08     15  2021-05-29 11:17:18  0.27-a0    

In [39]:
easiest_version = dcss_data.groupby(["Version"]).size().reset_index(name='counts').sort_values(by="counts", ascending=False)

In [40]:
easiest_version

,Version,counts
34,0.19-a0,97
44,0.21-a0,76
27,0.17-a0,76
54,0.23-a0,61
50,0.22-a0,51
...,...,...
16,0.14.2,1
10,0.13-b1,1
7,0.12-a0,1
5,0.11.0,1


In [42]:
easiest_version.to_csv("new.csv")